In [9]:
import os
import os.path
import sys
import pandas as pd
import numpy as np
from importlib import reload

In [27]:
LOCAL_PATH = 'data'
RAW_TRACE = 'bilibili.pcapng'
TCP_TRACE_FEATURE_FILE = 'bilibili_tcp_pkt.csv'
UDP_TRACE_FEATURE_FILE = 'bilibili_udp_pkt.csv'
TCP_FLOW_FEATURE_FILE = 'bilibili_tcp_flow.csv'
UDP_FLOW_FEATURE_FILE = 'bilibili_udp_flow.csv'
# BUCKET_NAME = '' # replace with your bucket name
# KEY = '' # replace with your object key

In [28]:
if not os.path.exists(os.path.join(LOCAL_PATH, RAW_TRACE)):
    if not os.path.exists(LOCAL_PATH):
        os.mkdir(LOCAL_PATH)
    
    import boto3
    import botocore

    s3 = boto3.resource('s3')

    try:
        s3.Bucket(BUCKET_NAME).download_file(KEY, os.path.join(LOCAL_PATH, RAW_TRACE))
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise

In [29]:
# convert raw trace to readable udp and tcp packet feature csv file
from python import packet_feature
reload(packet_feature)
%time packet_feature.udp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) , os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE))
%time packet_feature.tcp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) , os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE))

Conversion done
CPU times: user 19.5 ms, sys: 25.7 ms, total: 45.2 ms
Wall time: 11.9 s
Conversion done
CPU times: user 2.85 s, sys: 237 ms, total: 3.09 s
Wall time: 17.5 s


In [30]:
def to_csv(trace_df, extract_func, file, features, max_packets_per_flow):
    df = extract_func(trace_df, 1.0, upsampled=True, max_packets_per_flow=max_packets_per_flow)
    if df.shape[0] == 0:
        pd.DataFrame(columns=features).to_csv(file, index=False)
    else:
        df[features].to_csv(file, index=False)

In [31]:
from python import flow_feature
reload(flow_feature)

FEATURES = ['avg(pkt_len)', 'stddev(pkt_len)', 'fb_ratio', 'inter_arrival_time', 'pkt_count']

tcp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE))
udp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE))
for trace_df in [tcp_trace_df, udp_trace_df]:
    if tcp_trace_df is trace_df:
        to_csv(trace_df, flow_feature.tcp_generate, os.path.join(LOCAL_PATH, TCP_FLOW_FEATURE_FILE), FEATURES, max_packets_per_flow=1000)
    elif udp_trace_df is trace_df:
        to_csv(trace_df, flow_feature.udp_generate, os.path.join(LOCAL_PATH, UDP_FLOW_FEATURE_FILE), FEATURES, max_packets_per_flow=1000)
    else:
        raise

tcp flows, max pkt per flow->1000: 100%|██████████| 98181/98181 [00:31<00:00, 3137.04it/s]
/Users/shuojiang/anaconda2/envs/py36/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/Users/shuojiang/anaconda2/envs/py36/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
udp flows, max pkt per flow->1000: 100%|██████████| 28/28 [00:00<00:00, 4339.04it/s]
